In [14]:
!pip install nbformat importnb pandas matplotlib numpy tensorflow scikit-learn torch tensorflow_datasets transformers datasets resource

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# import tensorflow_datasets as tfds
# import os
# import re
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset, DatasetDict

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for JsonForm: filename=JsonForm-0.0.2-py3-none-any.whl size=3338 sha256=02a2ac734e4319deaf15a980ae8d745b4463777b133a839314328eb13180c91f
  Stored in directory: c:\users\saink\appdata\local\pip\cache\wheels\2e\bd\ab\c1026535edf314ce2b0d7ba3b2dd0ca67bfb5bae2cb301510f
  Created wheel for JsonSir: filename=JsonSir-0.0.2-py3-none-any.whl size=4775 sha256=0040f4821ef28a2b3c9f36837886c23be42d7e78dc3e402790e8cf6ea991ecc0
  Stored in directory: c:\users\saink\appdata\local\pip\cache\wheels\bb\e7\72\08831f4f1927bfcb155f0a971e253bfe677172cebea1332b51
Successfully built JsonForm JsonSir


In [1]:
# 라이블러리 불러오기
import os
import importlib
from importnb import Notebook
import pandas as pd
import json

import pandas as pd
from sklearn.model_selection import train_test_split

import torch


In [2]:
def file_import(folderName):
    # 현재 작업 디렉토리를 가져옵니다.
    current_dir = os.getcwd()
    models_dir = os.path.join(current_dir, f'{folderName}')

    # models 디렉토리 내의 모든 .ipynb 파일을 가져옵니다.
    notebook_files = [f for f in os.listdir(models_dir) if f.endswith('.ipynb')]

    # 모든 .ipynb 파일을 동적으로 import합니다.
    for notebook_file in notebook_files:
        module_name = notebook_file[:-6]  # .ipynb 확장자 제거
        module_path = f'{folderName}.{module_name}'

        with Notebook():
            module = importlib.import_module(module_path)

        # 가져온 모듈을 globals()에 추가합니다.
        globals()[module_name] = module


file_import('utills')
file_import('models')

c:\Users\saink\PycharmProjects\aiffel_DLthon01\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'resource'

In [3]:
# 전역 변수 설정
seed = 42

In [4]:
# 데이터 불러오기
train_data_path ="./data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

with open('./data/test.json') as f:
    test_data = json.load(f)

In [5]:
print(train_data.head())

   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [6]:
# 실험 셋 설정
expSet = [{'pre':'pre.preprocess_sentence',
           'train_test_split':{'test_size':0.2},
           'model':'klue_roberta_base'
          }
         ]

In [7]:
for exp in expSet:
    tmp = train_data.copy()
    # 전처리
    if 'pre' in exp:
        print('전처리')
        function_to_call = f'tmp["pre_"] = tmp["conversation"].apply(pre.preprocess_sentence)'
        exec(f'tmp["pre"] = {function_to_call}')
    # 데이터 분할
    if 'train_test_split' in exp:
        print('데이터 분할')
        train_df, test_df = train_test_split(tmp, test_size=exp['train_test_split']['test_size']
                                             , stratify=tmp['class'], random_state=seed)
    
    # 모델 불러오기
    print('모델 설정')
    x = exp['model']
    function_to_call = f'{x}.model(train_df, test_df)'
    exec(f'trainer = {function_to_call}')
#     model_function = globals()[exp['model']]
#     trainer = eval(f'{model_function}(train_df, test_df)')

전처리
데이터 분할
모델 설정


Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classif

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# klue_roberta_base.model()

for exp in expSet:
    x = exp['model']
    print(f'{x}.model(train_df, test_df)')

klue_roberta_base.model(train_df, test_df)


In [9]:
# !pip install --upgrade wandb

In [ ]:
import wandb
import os
os.environ["WANDB_API_KEY"] = "a8fa41ae061743cfb789129f6a363a02ff8ecebc"

# Initialize wandb
wandb.init(project="test")

# 모델 학습
trainer.train()

In [13]:
print(train_df.head())
print(test_df.head())

       idx        class                                       conversation  \
145    145    기타 괴롭힘 대화  야 못생긴놈\n뭐야 그만해\n얼레리꼴레리 너는 눈도 작고 코도작고 못생겼냐\n그만해...   
173    173  직장 내 괴롭힘 대화  이사원 이업무좀 내일까지 할수있나\n저 지금 하고있는거도있고 내일까지는 힘들거같습니...   
2364  2364        협박 대화  시끄러워 살수가 없어요\n우리집은 아이도 교육시켜 조용히 걸어요\n지금도 아이 소리...   
1316  1316  직장 내 괴롭힘 대화  김대리 이거 프로젝트 왜아직까지 완료가안된거지?\n과장님 프로젝트 다음주까지 연기되...   
608    608    기타 괴롭힘 대화  어이 택시 용호동가줘\n 네\n 아니 이쪽길말고 저리가야지\n 이쪽길이 빠릅니다 손...   

                                                    pre  \
145   야 못생긴놈 뭐야 그만해 얼레리꼴레리 너는 눈도 작고 코도작고 못생겼냐 그만해 선생...   
173   이사원 이업무좀 내일까지 할수있나 저 지금 하고있는거도있고 내일까지는 힘들거같습니다...   
2364  시끄러워 살수가 없어요 우리집은 아이도 교육시켜 조용히 걸어요 지금도 아이 소리 지...   
1316  김대리 이거 프로젝트 왜아직까지 완료가안된거지 ? 과장님 프로젝트 다음주까지 연기되...   
608   어이 택시 용호동가줘 네 아니 이쪽길말고 저리가야지 이쪽길이 빠릅니다 손님 하 . ...   

                                                   pre_  
145   야 못생긴놈 뭐야 그만해 얼레리꼴레리 너는 눈도 작고 코도작고 못생겼냐 그만해 선생...  
173   이사원 이업무좀 내일까지 할수있나 저 지금 하고있는거도있고 내일까지는 힘들거같습니다...  
23